In [37]:
import os
import cv2 
import importer
import numpy as np 
from img2bin import extract_img

In [112]:
NUMBER_OF_PICTURES = 1000
PICTURE_SIZE = 28

In [101]:
def read_img_file(dtype): 
    if dtype == 'train':
        RELEVANT_URL = './letters_train'
    elif dtype == 'test':
        RELEVANT_URL = './letters_test'
    fileList1 = [x for x in os.listdir(RELEVANT_URL)] 
    num_of_letters = len(fileList1) 
    distribute_table = [] 
    in_total = 0 
    for j in range(num_of_letters): 
        characterPath = os.path.join(RELEVANT_URL, fileList1[j]) 
        fileList2 = [x for x in os.listdir(characterPath)] 
        length = len(fileList2) 
        if length >= NUMBER_OF_PICTURES: 
            distribute = {'num_of_inputs': NUMBER_OF_PICTURES, 
                          'start_point': in_total}
            in_total = in_total + NUMBER_OF_PICTURES
        else: 
            distribute = {'num_of_inputs': length, 
                          'start_point': in_total}
            in_total = in_total + length
        distribute_table.append(distribute) 

        images = np.ndarray((in_total , PICTURE_SIZE * PICTURE_SIZE), np.float32) 
        images_label = np.ndarray(in_total, np.int32) 
        
    for j in range(num_of_letters): 
        characterPath = os.path.join(RELEVANT_URL, fileList1[j]) 
        fileList2 = [x for x in os.listdir(characterPath)] 

        for i in range(distribute_table[j]['num_of_inputs']): 
            img = cv2.imread(os.path.join(characterPath, fileList2[i]))
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)  
            ret, binary = cv2.threshold(gray,149,1,cv2.THRESH_BINARY)  
            img, contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) 
            contoured_imgs = []
            for contour in contours:
                location = cv2.boundingRect(contour)
                #print(location)
                x, y, w, h = location
                if(w*h<100):
                    continue
                extracted_img = extract_img(location,img,contour)
                contoured_imgs.append(extracted_img)
            if len(contoured_imgs):
                img = contoured_imgs[0].reshape(28*28) 
                t = np.ones(len(img), np.uint8)
                for _j in range(len(img)):
                    if img[_j] == True:
                        t[_j] = 255
                    else:
                        t[_j] = 0
                img = t
            else:
                img = binary.reshape(28*28)
                for _i in range(len(img)):
                    if img[_i] == 1:
                        img[_i] = 0
                    else:
                        img[_i] = 255
            images[distribute_table[j]['start_point'] + i] = img
            images_label[distribute_table[j]['start_point'] + i] = int(fileList1[j]) - 1

    return images, images_label 

输入图片中将字迹处记为0，背景记为1；读文件夹列表时顺序是1,10,11...和平时的顺序不同;EMNIST数据集中的图片都是以块的方式排列的，需要提取字母的边缘才能和输入的形式统一。